# Init

In [ ]:
import common

common.GrConf() #build common scope config
productsCaller = common.GrGetAPI('Objects','products')
productsCaller.buildParam("active=1")

# Workflow to import Grocy product data
Init gets everything ready for product calls. It'll call it next

In [ ]:
productsCaller.get()
r=productsCaller.r
if not "200" in str(r):
    print(f'Call failed :( Error:{str(r)}')
else:
    print('Success')

# Tabulate Data
Input response to DataFrame & Display as an ITable

In [ ]:
import pandas as pd
import io
from itables import init_notebook_mode
from itables import show

rjson = r.json()
callThese = []
columnMap = {}
alias=common.CONF.build_alias(rjson)
boolean = common.CONF.build_bools(rjson)
if "userfields" in rjson[0].keys():
    for i in range(len(rjson)):
        custfields = rjson[i].pop("userfields")
        for field in custfields:
            rjson[i][field] = custfields[field]
productsFrame = pd.DataFrame(rjson)
for column in productsFrame.columns.values.tolist():
    if column in alias.keys():
        # there is an id alias mapping
        # condition to string
        # conditioning leaves a smell - string value <NA> which is not displayed and not recognized by isna
        productsFrame[column] = productsFrame[column].astype('Int64')
        productsFrame[column] = productsFrame[column].astype('str')
        productsFrame[column] = productsFrame[column].fillna("")
        for definition in alias[column]:
            for ident in definition:
                productsFrame.loc[productsFrame[column] == str(ident), column] = str(definition[ident])
    if column in boolean:
        #map binary to bool
        productsFrame[column] = productsFrame[column].astype('bool')

init_notebook_mode(all_interactive=True)
show(productsFrame,buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

In [ ]:
# filter by query example
# use user input for column and value
# That <NA> smell is on display here
show(productsFrame.loc[productsFrame['product_group_id'] == '<NA>'],buttons=["copyHtml5", "csvHtml5", "excelHtml5"])
dfbatchupload = productsFrame.loc[productsFrame['product_group_id'] == '<NA>']
#hack for userfields. Probably should expand these in bootstrap based on whats currently created?
#dfbatchupload.loc[:,'userfields'] = dfbatchupload['userfields'].astype('str')
dfbatchupload = dfbatchupload.replace("<NA>","")

In [ ]:
#filter by substring example
#use user input for column and value
show(productsFrame[productsFrame['name'].str.contains('ANY')],buttons=["copyHtml5", "csvHtml5", "excelHtml5"])

# Load to mass load spreadsheet
Make any filterations in place in productsFrame, name it dfbatchupload then call the next cell to load the list into a mass put spreadsheet. Then call the mass put script if you want to make a mass change to the results

In [ ]:
batchuploaddata = dfbatchupload.values.tolist()
creds = common.GoogleOAuth()
sheetname = 'products'
spreadsheetId = common.CONF.sheets['BatchUpload']['spreadsheetId']
#clear anything left in there
clearLastPost = common.sheets_clear(creds,spreadsheetId,sheetname,'A3:ZZ1000')
#add the results to the tab
common.sheets_append(creds,spreadsheetId,sheetname,batchuploaddata,'A2')
#optimize columns
updater = common.GoBatchUpdate(creds,spreadsheetId)
optistyle = common.GrStyleGuide('request_styles','optimize_columns')
request = optistyle.guide
request['autoResizeDimensions']['dimensions']['sheetId'] = common.CONF.sheets['BatchUpload']['sheets']['products']
request['autoResizeDimensions']['dimensions']['endIndex'] = 100
updater.addReq(request)
#add request to bring the sheet forward
request = {"updateSheetProperties": {
        "properties": {
            "sheetId": common.CONF.sheets['BatchUpload']['sheets']['products'],
            "index":0
        },
        "fields": "index"
}
          }
updater.addReq(request)
updater.call()